In [2]:
def fatcatproc_part1(subjdir, sesdir):
    sourcedir = Path(sesdir,'dti')
    subjroot = "_".join([subjdir.name, sesdir.name])

    #Creating processing directories
    afniprocdir = Path(sourcedir, 'afni')
    afniprocdir.mkdir(exist_ok=True)
    fatcat_dir = Path(afniprocdir, 'fatcat')
    fatcat_dir.mkdir(exist_ok=True)
    fatcat_dwidir = fatcat_dir / 'dwi'
    fatcat_dwidir.mkdir(exist_ok=True)
    fatcat_t2wdir = fatcat_dir / 't2w'
    fatcat_t2wdir.mkdir(exist_ok=True)
    fatcat_t1wdir = fatcat_dir / 't1w'
    fatcat_t1wdir.mkdir(exist_ok=True)

    #Creating source file variable
    sourcedwi = sourcedir / (subjroot + "_ppd.mif")

    #Creating mrconvert output file variables
    fatcat_bvec = fatcat_dwidir / (subjroot + ".bvec")
    fatcat_bval = fatcat_dwidir / (subjroot + ".bval")
    fatcat_dwinii = fatcat_dwidir / (subjroot + ".nii")

    #Running mrconvert on sourcedwi and outputting to fatcat_dwidir
    !mrconvert -force -export_grad_fsl $fatcat_bvec $fatcat_bval $sourcedwi $fatcat_dwinii

    #>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>#
    #-------fat_proc_convert_dcm_dwis--------#
    #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<#
    os.chdir(fatcat_dir) #Must change to fatcat_dir first
    !fat_proc_convert_dcm_dwis -innii $fatcat_dwinii -inbvec $fatcat_bvec -inbval $fatcat_bval -prefix $fatcat_dir/dwi_00/dwi

    #>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>#
    #-------Copying original T1w and T2w volumes--------#
    #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<#

    #Creating variables as necessary to copy the T2w and T1w volumes from BIDS_master
    ##Source variables
    anat_sourcedir = Path(bidsmaster_dir, subjdir.name, sesdir.name, 'anat')
    orig_t1w = Path(anat_sourcedir, "_".join([subjdir.name, sesdir.name, 'acq-AXFSPGRBRAVONEW', 'T1w.nii']))
    orig_t2w = Path(anat_sourcedir, "_".join([subjdir.name, sesdir.name, 'acq-AxT2FLAIRCOPYDTI', 'T2w.nii']))

    ##Destination variables
    fatcat_t1w = fatcat_t1wdir / 't1w.nii'
    fatcat_t2w = fatcat_t2wdir / 't2w.nii'

    #Copying original T1w and T2w volumes to fatcat/t1w and fatcat/t2w
    if anat_sourcedir.exists():
        print("Copying T1w NiFTI...")  
        shutil.copy(orig_t1w, fatcat_t1w)

        print("Copying T2w NiFTI...")
        shutil.copy(orig_t2w, fatcat_t2w)

    #>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>#
    #-------fat_proc_convert_dcm_anat--------#
    #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<#

    #Making output directory
    fatcat_anat00dir = fatcat_dir / 'anat_00'

    #T1w - Outputting to fatcat_anat00dir/t1w.nii.gz
    !fat_proc_convert_dcm_anat -innii $fatcat_t1w -prefix $fatcat_anat00dir/t1w
    fatcat_anat00dir_t1w = fatcat_anat00dir / 't1w.nii.gz'

    #T2w - Outputting to fatcat_anat00dir/t2w.nii.gz
    !fat_proc_convert_dcm_anat -innii $fatcat_t2w -prefix $fatcat_anat00dir/t2w
    fatcat_anat00dir_t2w = fatcat_anat00dir / 't2w.nii.gz'

    #>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>#
    #-------fat_proc_axialize_anat--------#
    #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<#

    #Making output directory
    fatcat_anat01dir = fatcat_dir / 'anat_01'

    #Axializing the T2w volume - Outputting to fatcat_anat00dir/t2w.nii.gz
    !fat_proc_axialize_anat -inset $fatcat_anat00dir_t2w -prefix $fatcat_anat01dir/t2w -mode_t2w \
    -refset $axializeref_t2w -extra_al_wtmask $axializeref_t2w_wt -out_match_ref -extra_al_opts "-newgrid 1.0"
    fatcat_anat01dir_t2w = fatcat_anat01dir / 't2w.nii.gz'

    #>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>#
    #-------fat_proc_axialize_anat--------#
    #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<#

    !align_epi_anat.py -dset1 $fatcat_anat01dir_t2w -dset2 $fatcat_anat00dir_t1w -align_centers yes -dset1_strip None \
    -dset2_strip None -big_move -rigid_body -dset2to1 -dset2_base 0 -prep_off -suffix align


In [ ]:
import os,sys,shutil
from pathlib import Path

bidsproc_dir = Path('/scratch/jdrussell3/bidsproc')
bidsmaster_dir = Path('/Volumes/Vol6/YouthPTSD/BIDS_master/')

axializeref_t2w = Path('/scratch/jdrussell3/mni2009/mni_icbm152_t2_relx_tal_nlin_sym_09a_ACPCE.nii.gz')
axializeref_t2w_wt = Path('/scratch/jdrussell3/mni2009/mni_icbm152_t1_tal_nlin_sym_09a_MSKD_ACPCE_wtell.nii.gz')

subjdirs = (subjdir for subjdir in bidsproc_dir.iterdir() if subjdir.is_dir())

%env OMP_NUM_THREADS=32

for subjdir in sorted(subjdirs):
    sesdirs = (sesdir for sesdir in subjdir.iterdir() if sesdir.is_dir())
    for sesdir in sorted(sesdirs):
        dtidir = sesdir/'dti'
        if dtidir.exists():
            try:
                fatcatproc_part1(subjdir, sesdir)
            except Exception as e:
                print(e)
                errfile = Path(bidsproc_dir / 'fatcatproc_pt1_error_log.txt')
                with open(errfile, 'a+') as errorfile:
                    errorfile.write(str('Error processing ' + subjdir.name + ', ' + \
                                        sesdir.name + ':\n' + str(e) + '\n'))
                next

env: OMP_NUM_THREADS=32
mrconvert: [WARNING] existing output files will be overwritten
mrconvert: [100%] copying from "/scratch/j.../dti/sub-001_ses-01_ppd.mif" to "/scratch/j...tcat/dwi/sub-001_ses-01.nii"
++ User has listed 1 nifti dsets
++ User has listed 1 bvec sets
++ User has listed 1 bval sets

++ Based on prefix, the output directory will be:
     /scratch/jdrussell3/bidsproc/sub-001/ses-01/dti/afni/fatcat/dwi_00
++ Based on prefix, the output prefix will be:
     dwi

+* WARNING: Somehow found a premade working directory (?):
     /scratch/jdrussell3/bidsproc/sub-001/ses-01/dti/afni/fatcat/dwi_00/__WORKING_convert_dcm_dwis_dwi
   NB: will *not* clean it afterwards.

-----> STARTING fat_proc_convert_dcm_dwis ---->


This command:
fat_proc_convert_dcm_dwis -innii /scratch/jdrussell3/bidsproc/sub-001/ses-01/dti/afni/fatcat/dwi/sub-001_ses-01.nii -inbvec /scratch/jdrussell3/bidsproc/sub-001/ses-01/dti/afni/fatcat/dwi/sub-001_ses-01.bvec -inbval /scratch/jdrussell3/bidsproc/sub-001

++ Output dataset /scratch/jdrussell3/bidsproc/sub-001/ses-01/dti/afni/fatcat/anat_00/__WORKING_convert_dcm_anat_t2w/_init_vol.nii.gz
++ Using nifti_tool to remove ID info, if possible.
+d stripped extras from 1 files
0.760902  -22.2728  15.8619
*+ WARNING: Over-writing dataset ../t2w.nii.gz
++ Wrote new dataset: ../t2w.nii.gz

All done!

Your output anatomical is:
	/scratch/jdrussell3/bidsproc/sub-001/ses-01/dti/afni/fatcat/anat_00/t2w.nii.gz




++ QC image 00 (/scratch/jdrussell3/bidsproc/sub-001/ses-01/dti/afni/fatcat/anat_00/t2w.nii.gz): t2w_qc_anat



++ My command:
   @chauffeur_afni -ulay /scratch/jdrussell3/bidsproc/sub-001/ses-01/dti/afni/fatcat/anat_00/t2w.nii.gz -ulay_range 2% 98% -olay_off -prefix /scratch/jdrussell3/bidsproc/sub-001/ses-01/dti/afni/fatcat/anat_00/QC/t2w_qc_anat -montx 5 -monty 3 -set_xhairs OFF -label_mode 1 -label_size 3 -do_clean

------------------ start of optionizing ------------------

++ Found input file:   /scratch/jdrussell3/bidsproc/sub-001/ses-

++ Output dataset /scratch/jdrussell3/bidsproc/sub-001/ses-01/dti/afni/fatcat/anat_01/__WORKING_axialize_anat_t2w/tt_00.nii
++ Distance between input's center of mass to (0,0,0) is 0.00000 mm.
   -> if this is large, you might want to recenter with:
        3dCM -set 0 0 0 FILE
      or something (you could copy it first).
++ Doing a little anisosmoothing.

+++ Command Echo:
   3danisosmooth -overwrite -iters 2 -prefix tt_01_ani.nii -3D tt_00.nii 

++ 3danisosmooth: AFNI version=AFNI_19.2.07 (Jul 29 2019) [64-bit]
++ Authored by: Daniel Glen
++ loading original data
++ iteration 0
++    computing structure tensor
++     applying structure tensor
++ iteration 1
++    computing structure tensor
++     applying structure tensor
++ --- Output dataset ./tt_01_ani.nii
++ Executing 3dcalc()
 argv[0]=3dcalc argv[1]=-a argv[2]=tt_01_ani.nii argv[3]=-expr argv[4]=step(a) argv[5]=-session argv[6]=/tmp argv[7]=-prefix argv[8]=3dcalc_AFN_tbamIhFu9M-27CqK2JFhqw argv[9]=-verbose
++ 3dcalc: AFNI versi

++ Output dataset ./tt_09_alnd0.nii
++ 3dAllineate: total CPU time = 0.0 sec  Elapsed = 4.0
++ ###########################################################

+++ Command Echo:
   3dcalc -overwrite -a tt_09_alnd0.nii -expr a -prefix ../t2w.nii.gz 

++ 3dcalc: AFNI version=AFNI_19.2.07 (Jul 29 2019) [64-bit]
++ Authored by: A cast of thousands
++ Output dataset ./../t2w.nii.gz
++ Making QC images.
++ QC image 00 (final output ulay): t2w_qc_xlzd
++ My command:
   @chauffeur_afni -ulay ../t2w.nii.gz -prefix ../QC/t2w_qc_xlzd -montx 3 -monty 3 -set_xhairs MULTI -label_mode 1 -label_size 3 -do_clean

------------------ start of optionizing ------------------

++ Found input file:   ../t2w.nii.gz
+* Noting: you did NOT provide an overlay, and yet
           you did NOT turn off the overlay with '-olay_off'
       --> so I assume you just want to view the underlay.
++ Using blowup factor: 2
++ Using opacity:  6
++ Making temporary work directory to copy vis files: ../QC/__tmp_chauf_VRYuUwag5Go
+

#Script is running (command trimmed):
  3dinfo ./__tt_t2w+orig | \grep 'Data Axes Tilt:'|\grep 'Oblique'
#++ Dataset /scratch/jdrussell3/bidsproc/sub-001/ses-01/dti/afni/fatcat/__tt_t2w+orig is not oblique
#Script is running (command trimmed):
  3dinfo ./anat_00/t1w.nii.gz | \grep 'Data Axes Tilt:'|\grep 'Oblique'
#++ Dataset /scratch/jdrussell3/bidsproc/sub-001/ses-01/dti/afni/fatcat/anat_00/t1w.nii.gz is not oblique
#Script is running (command trimmed):
  3dAttribute DELTA ./__tt_t2w+orig
#++ Spacing for dset1 to oblique dset2 alignment is 1.000000
#Script is running (command trimmed):
  3dcopy ./__tt_t2w+orig ./__tt_t2w_ob; @Align_Centers -base ./anat_00/t1w.nii.gz -dset ./__tt_t2w_ob+orig -no_cp
++ 3dcopy: AFNI version=AFNI_19.2.07 (Jul 29 2019) [64-bit]
++ 3drefit: AFNI version=AFNI_19.2.07 (Jul 29 2019) [64-bit]
++ Authored by: RW Cox
++ Processing AFNI dataset ./__tt_t2w_ob+orig
 + deoblique
++ 3drefit processed 1 datasets
#Script is running (command trimmed):
  cat_matvec ./__t

...

++ master dataset for output = source
++ OpenMP thread count = 32
++ ========== Applying transformation to 1 sub-bricks ==========
++ wsinc5 interpolation setup:
 +   taper function  = Min sidelobe 3 term
 +   taper cut point = 0.000
 +   window radius   = 5 voxels
 +   window shape    = Cubical
 +   The above can be altered via the AFNI_WSINC5_* environment variables.
 +  (To avoid this message, 'setenv AFNI_WSINC5_SILENT YES'.)
 +   wsinc5 CUBE(5) mask has 1000 points
++ Output dataset ./tt_06_LRmid.nii
++ 3dAllineate: total CPU time = 0.0 sec  Elapsed = 1.1
++ ###########################################################
++ New inp0 = tt_07_origLR.nii

+++ Command Echo:
   3dAllineate -1Dmatrix_apply _map_lr_half.aff12.1D -prefix tt_07_origLR.nii -source tt_00.nii -final wsinc5 -overwrite 

++ 3dAllineate: AFNI version=AFNI_19.2.07 (Jul 29 2019) [64-bit]
++ Authored by: Zhark the Registrator
++ Source dataset: ./tt_00.nii
++ Base dataset:   (not given)
++ You might want to use '-maste

++ Writing one 1152x1416 image to filter '/usr/bin/pnmtopng -compression 9 > ../QC/t2w_qc_xlzd.axi.png'
pnmtopng: 154 colors found

AFNI QUITTs!

+* Removing temporary image directory '../QC/__tmp_chauf_z8c6ofg8uFr'.

[1]  + Done                          Xvfb :134 -screen 0 1024x768x24
++ QC image 01 (earlier allin olaying ref ulay): t2w_qc_ref_in
++ 3dedge3: AFNI version=AFNI_19.2.07 (Jul 29 2019) [64-bit]
++ My command:
   @chauffeur_afni -ulay REF.nii -ulay_range 2% 98% -olay ./tt_alnd_edges.nii -func_range_perc_nz 50 -pbar_posonly -cbar red_monochrome -opacity 6 -prefix ../QC/t2w_qc_ref_in -montx 3 -monty 3 -set_xhairs MULTI -label_mode 1 -label_size 3 -do_clean

------------------ start of optionizing ------------------

++ Found input file:   REF.nii
++ Found input file:   ./tt_alnd_edges.nii
++ Using blowup factor: 2
++ Using opacity:  6
++ Making temporary work directory to copy vis files: ../QC/__tmp_chauf_ajZHHa79o4w
++ Converted 0 to labels
++ Final subbrick indices: -1 -1 -

++ 3dcalc: AFNI version=AFNI_19.2.07 (Jul 29 2019) [64-bit]
++ Authored by: A cast of thousands
++ Output dataset /scratch/jdrussell3/bidsproc/sub-011/ses-02/dti/afni/fatcat/anat_00/__WORKING_convert_dcm_anat_t1w/_init_vol.nii.gz
++ Using nifti_tool to remove ID info, if possible.
+d stripped extras from 1 files
-3.19682  -25.0111  33.3301
*+ WARNING: Over-writing dataset ../t1w.nii.gz
++ Wrote new dataset: ../t1w.nii.gz

All done!

Your output anatomical is:
	/scratch/jdrussell3/bidsproc/sub-011/ses-02/dti/afni/fatcat/anat_00/t1w.nii.gz




++ QC image 00 (/scratch/jdrussell3/bidsproc/sub-011/ses-02/dti/afni/fatcat/anat_00/t1w.nii.gz): t1w_qc_anat



++ My command:
   @chauffeur_afni -ulay /scratch/jdrussell3/bidsproc/sub-011/ses-02/dti/afni/fatcat/anat_00/t1w.nii.gz -ulay_range 2% 98% -olay_off -prefix /scratch/jdrussell3/bidsproc/sub-011/ses-02/dti/afni/fatcat/anat_00/QC/t1w_qc_anat -montx 5 -monty 3 -set_xhairs OFF -label_mode 1 -label_size 3 -do_clean

------------------ start of 

Initializing: X11[The X.Org Foundation v 11906000].. Widgets...... Input files:+/scratch/jdrussell3/bidsproc/sub-011/ses-02/dti/afni/fatcat/anat_00/QC/__tmp_chauf_DvJpfeaMbuY
 session #  1  = QC/__tmp_chauf_DvJpfeaMbuY/ ==> 1 dataset 
 dataset count = 1
 Time series   = 0 files read
 Plugins       = 0 libraries read
++ Writing one 5120x2094 image to filter '/usr/bin/pnmtopng -compression 9 > /scratch/jdrussell3/bidsproc/sub-011/ses-02/dti/afni/fatcat/anat_00/QC/t2w_qc_anat.sag.png'
pnmtopng: 153 colors found
++ Writing one 5120x2094 image to filter '/usr/bin/pnmtopng -compression 9 > /scratch/jdrussell3/bidsproc/sub-011/ses-02/dti/afni/fatcat/anat_00/QC/t2w_qc_anat.cor.png'
pnmtopng: 153 colors found
++ Writing one 5120x3072 image to filter '/usr/bin/pnmtopng -compression 9 > /scratch/jdrussell3/bidsproc/sub-011/ses-02/dti/afni/fatcat/anat_00/QC/t2w_qc_anat.axi.png'
pnmtopng: 153 colors found

AFNI QUITTs!

+* Removing temporary image directory '/scratch/jdrussell3/bidsproc/sub-011/ses

++ master dataset for output = source
++ OpenMP thread count = 32
++ ========== Applying transformation to 1 sub-bricks ==========
++ wsinc5 interpolation setup:
 +   taper function  = Min sidelobe 3 term
 +   taper cut point = 0.000
 +   window radius   = 5 voxels
 +   window shape    = Cubical
 +   The above can be altered via the AFNI_WSINC5_* environment variables.
 +  (To avoid this message, 'setenv AFNI_WSINC5_SILENT YES'.)
 +   wsinc5 CUBE(5) mask has 1000 points


In [17]:
###Converting misplced post-align_epi_anat.py BRIK/HEADER files to NIFTI 
###and moving them to anat_01

import os,sys,shutil,gzip,glob
from pathlib import Path

bidsproc_dir = Path('/scratch/jdrussell3/bidsproc')

subjdirs = (subjdir for subjdir in bidsproc_dir.iterdir() if subjdir.is_dir())

for subjdir in sorted(subjdirs):
    sesdirs = (sesdir for sesdir in subjdir.iterdir() if sesdir.is_dir())
    for sesdir in sorted(sesdirs):
        dtidir = sesdir/'dti'
        if dtidir.exists():
            fatcat_dir = dtidir / 'afni' / 'fatcat'
        else:
            next
        if fatcat_dir.exists():
            fatcat_anat01dir = dtidir / 'afni' / 'fatcat' / 'anat_01'
            fatcat_anat01dir_t1wprefix = fatcat_anat01dir / 't1w'
            fatcat_anat01dir_t1wnii = fatcat_anat01dir / 't1w.nii'
            fatcat_anat01dir_t1w = fatcat_anat01dir / 't1w.nii.gz'
            os.chdir(fatcat_dir)
            !3dAFNItoNIFTI -prefix $fatcat_anat01dir_t1wprefix 't1walign+orig'
            with open(fatcat_anat01dir_t1wnii, 'rb') as f_in:
                with gzip.open(fatcat_anat01dir_t1w, 'wb') as f_out:
                    shutil.copyfileobj(f_in, f_out)
            for tocopy in fatcat_dir.glob('t1w*.1D'):
                shutil.copy(tocopy, fatcat_anat01dir)
                
            for todel in fatcat_dir.iterdir():
                if todel.is_file():
                    os.remove(todel)

        

++ 3dAFNItoNIFTI: AFNI version=AFNI_19.2.07 (Jul 29 2019) [64-bit]
++ 3dAFNItoNIFTI: AFNI version=AFNI_19.2.07 (Jul 29 2019) [64-bit]
** FATAL ERROR: Can't open dataset 't1walign+orig'
** Program compile date = Jul 29 2019
++ 3dAFNItoNIFTI: AFNI version=AFNI_19.2.07 (Jul 29 2019) [64-bit]
++ 3dAFNItoNIFTI: AFNI version=AFNI_19.2.07 (Jul 29 2019) [64-bit]
++ 3dAFNItoNIFTI: AFNI version=AFNI_19.2.07 (Jul 29 2019) [64-bit]
++ 3dAFNItoNIFTI: AFNI version=AFNI_19.2.07 (Jul 29 2019) [64-bit]
++ 3dAFNItoNIFTI: AFNI version=AFNI_19.2.07 (Jul 29 2019) [64-bit]
++ 3dAFNItoNIFTI: AFNI version=AFNI_19.2.07 (Jul 29 2019) [64-bit]
++ 3dAFNItoNIFTI: AFNI version=AFNI_19.2.07 (Jul 29 2019) [64-bit]
++ 3dAFNItoNIFTI: AFNI version=AFNI_19.2.07 (Jul 29 2019) [64-bit]
++ 3dAFNItoNIFTI: AFNI version=AFNI_19.2.07 (Jul 29 2019) [64-bit]
++ 3dAFNItoNIFTI: AFNI version=AFNI_19.2.07 (Jul 29 2019) [64-bit]
++ 3dAFNItoNIFTI: AFNI version=AFNI_19.2.07 (Jul 29 2019) [64-bit]
++ 3dAFNItoNIFTI: AFNI version=AFNI_19.2

++ 3dAFNItoNIFTI: AFNI version=AFNI_19.2.07 (Jul 29 2019) [64-bit]
++ 3dAFNItoNIFTI: AFNI version=AFNI_19.2.07 (Jul 29 2019) [64-bit]
++ 3dAFNItoNIFTI: AFNI version=AFNI_19.2.07 (Jul 29 2019) [64-bit]
++ 3dAFNItoNIFTI: AFNI version=AFNI_19.2.07 (Jul 29 2019) [64-bit]
++ 3dAFNItoNIFTI: AFNI version=AFNI_19.2.07 (Jul 29 2019) [64-bit]
++ 3dAFNItoNIFTI: AFNI version=AFNI_19.2.07 (Jul 29 2019) [64-bit]
++ 3dAFNItoNIFTI: AFNI version=AFNI_19.2.07 (Jul 29 2019) [64-bit]
++ 3dAFNItoNIFTI: AFNI version=AFNI_19.2.07 (Jul 29 2019) [64-bit]
++ 3dAFNItoNIFTI: AFNI version=AFNI_19.2.07 (Jul 29 2019) [64-bit]
++ 3dAFNItoNIFTI: AFNI version=AFNI_19.2.07 (Jul 29 2019) [64-bit]
++ 3dAFNItoNIFTI: AFNI version=AFNI_19.2.07 (Jul 29 2019) [64-bit]
++ 3dAFNItoNIFTI: AFNI version=AFNI_19.2.07 (Jul 29 2019) [64-bit]
++ 3dAFNItoNIFTI: AFNI version=AFNI_19.2.07 (Jul 29 2019) [64-bit]
++ 3dAFNItoNIFTI: AFNI version=AFNI_19.2.07 (Jul 29 2019) [64-bit]
++ 3dAFNItoNIFTI: AFNI version=AFNI_19.2.07 (Jul 29 2019) [64-

In [27]:
###Copying aligned T1w to 'toFS_dir' for FreeSurfer processing and
###deleting a lingering T1w.nii left over in the anat_01 directory

import os,sys,shutil,gzip,glob
from pathlib import Path

bidsproc_dir = Path('/scratch/jdrussell3/bidsproc')
toFS_dir = Path('/scratch/jdrussell3/toFS')
if not toFS_dir.exists():
    toFS_dir.mkdir()

subjdirs = (subjdir for subjdir in bidsproc_dir.iterdir() if subjdir.is_dir())

for subjdir in sorted(subjdirs):
    sesdirs = (sesdir for sesdir in subjdir.iterdir() if sesdir.is_dir())
    for sesdir in sorted(sesdirs):
        dtidir = sesdir/'dti'
        if dtidir.exists():
            fatcat_dir = dtidir / 'afni' / 'fatcat'
        else:
            next
        if fatcat_dir.exists():
            fatcat_anat01dir = fatcat_dir / 'anat_01'
            fatcat_anat01dir_t1wnii = fatcat_anat01dir / 't1w.nii'
            if fatcat_anat01dir_t1wnii.exists():
                os.remove(fatcat_anat01dir_t1wnii)
            fatcat_anat01dir_t1w = fatcat_anat01dir / 't1w.nii.gz'
            fatcat_toFS_t1w = toFS_dir / "_".join([subjdir.name, sesdir.name, 't1w.nii.gz'])
            shutil.copy(fatcat_anat01dir_t1w, fatcat_toFS_t1w)
            